In [1]:
import torch
from transformers import BertModel, BertTokenizer

In [2]:
from transformers import BertForSequenceClassification
model_path = "finetuned_models/model"
model = BertForSequenceClassification.from_pretrained(model_path)


In [48]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_path)


OSError: Can't load tokenizer for 'finetuned_models/model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'finetuned_models/model' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [3]:
class BERTSUM(torch.nn.Module):
    def __init__(self):
        super(BERTSUM, self).__init__()
        #self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.bert = BertForSequenceClassification.from_pretrained(model)

        self.dropout = torch.nn.Dropout(0.1)
        self.linear = torch.nn.Linear(self.bert.config.hidden_size, 1)
        #self.linear = torch.nn.Linear(self.bert.config.hidden_size, len(filtered_sentences))
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        scores = self.linear(pooled_output)
        return scores

In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
# Initialize the lemmatizer and stopword list
lemmatizer = WordNetLemmatizer()
stopwords = set(stopwords.words('english'))

# Original sentences
original_sentences = ["The quick brown fox jumps over the lazy dog.",
                       "This is a sentence.",
                       "Another sentence here.",
                       "This is a short sentence.",
                       "The dog is lazy."]

# Preprocess the sentences
filtered_sentences = []
for sentence in original_sentences:
    # Tokenize the sentence
    words = nltk.word_tokenize(sentence.lower())
    
    # Remove stopwords and lemmatize the remaining words
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords]
    
    # Convert the filtered words back to a sentence and add it to the list of filtered sentences
    filtered_sentence = ' '.join(filtered_words)
    filtered_sentences.append(filtered_sentence)

In [5]:
def bertsum_extractive_summarization_batch(original_sentences, filtered_sentences, batch_size=10, size=5):
    # Set the random seed for PyTorch
    torch.manual_seed(42)

    # Tokenize original sentences
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    # Initialize the BERTSUM model
    model = BERTSUM()
    model.eval()

    num_batches = len(filtered_sentences) // batch_size + int(len(filtered_sentences) % batch_size > 0)
    all_scores = []

    for batch_idx in range(num_batches):
        # Tokenize filtered sentences in the current batch
        batch_start = batch_idx * batch_size
        batch_end = min((batch_idx + 1) * batch_size, len(filtered_sentences))
        input_ids_filtered = tokenizer(filtered_sentences[batch_start:batch_end], padding=True, truncation=True, return_tensors='pt').input_ids
        attention_mask_filtered = tokenizer(filtered_sentences[batch_start:batch_end], padding=True, truncation=True, return_tensors='pt').attention_mask

        # Generate BERTSUM scores for filtered sentences in the current batch
        scores = model(torch.tensor(input_ids_filtered), torch.tensor(attention_mask_filtered))

        # Store scores for the current batch
        all_scores.extend(scores.view(-1).tolist())

    # Sort scores in descending order
    sorted_indices = sorted(range(len(all_scores)), key=lambda i: all_scores[i], reverse=True)

    # Select top "size" sentences with highest scores
    top_indices = sorted_indices[:size]
    # Generate final summary
    summary = ""
    for index in top_indices:
        summary += original_sentences[index] + " "

    return summary


In [ ]:
summary = bertsum_extractive_summarization_batch(original_sentences, filtered_sentences, size = 3)
summary

In [46]:
def bertsum_extractive_summarization(filtered_sentences, original_sentences, size=5):
    # Tokenize original sentences
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    input_ids_original = tokenizer(original_sentences, padding=True, truncation=True, return_tensors='pt').input_ids
    attention_mask_original = tokenizer(original_sentences, padding=True, truncation=True, return_tensors='pt').attention_mask

    # Tokenize filtered sentences
    input_ids_filtered = tokenizer(filtered_sentences, padding=True, truncation=True, return_tensors='pt').input_ids
    attention_mask_filtered = tokenizer(filtered_sentences, padding=True, truncation=True, return_tensors='pt').attention_mask

    # Initialize the BERTSUM model
    model = BERTSUM()
    model.eval()

    # Generate BERTSUM scores for filtered sentences
    scores = model(torch.tensor(input_ids_filtered), torch.tensor(attention_mask_filtered))

    # Sort scores in descending order
    sorted_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)

    # Select top "size" sentences with highest scores
    top_indices = sorted_indices[:size]

    # Generate final summary
    summary = ""
    for index in top_indices:
        summary += original_sentences[index] + " "

    return summary


In [47]:
original_sentences = ["The quick brown fox jumps over the lazy dog.", "This is a sentence.", "Another sentence here.", "This is a short sentence.", "The dog is lazy."]
filtered_sentences = ["The quick brown fox jumps over the lazy dog.", "Another sentence here.", "The dog is lazy."]
summary = bertsum_extractive_summarization(filtered_sentences, original_sentences, size=2)
print(summary)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

IndexError: tuple index out of range

In [ ]:
##self.linear = torch.nn.Linear(self.bert.config.hidden_size, 1)